<h1 align="center"> Computación Cientifica - Laboratorio 3: Simulando Númericamente Sistemas Dinámicos </h1>
<br>
<br>
** Autores **:
+ Yerson Escobar - 201273084-8 
+ Eduardo Ramírez - 201103031-1


## Introducción

## Desarrollo

### Simulando sistemas caóticos

### Módulos y funciones necesarias

In [51]:
import numpy as np
from scipy import integrate
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import cnames
from matplotlib import animation

### Función para visualizar resultados

In [52]:
"""
-> x_t
   arreglo con las posiciones (x,y,z) de las trayectorias. Este puede ser
   de dos dimensiones (len(t), 3) para una sola trayectoria, o puede ser 
   de tres dimensiones (len(t), 3, N_trajectories) para mas de una trayectoria
-> N_trajectories
   numero de trayectorias a visualizar
-> xlim,ylim,zlim
   limites en los ejes x,y,z de la animacion
-> rotate
   rotar mientras anima para mejores perspectivas
"""
def animate_lorenz(x_t, N_trajectories, xlim=(-20,70), ylim=(-35,35), zlim=(-35,35), rotate=False):
    #setting it to correct format
    if x_t.ndim==2:
        x_t = np.array([x_t])
    elif x_t.ndim==3: 
        x_t = np.rot90(x_t).T
    else:
        return -1
    #setting the number of frames
    frames = max(x_t.shape)
    frames /= 2
    #set up figure & 3D axis for animation
    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1], projection='3d')
    #ax.axis('off')

    #choose a different color for each trajectory
    colors = plt.cm.jet(np.linspace(0, 1, N_trajectories))

    #set up lines and points
    lines = sum([ax.plot([], [], [], '-', c=c)
                 for c in colors], [])
    pts = sum([ax.plot([], [], [], 'o', c=c)
               for c in colors], [])

    #prepare the axes limits
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_zlim(zlim)

    #set point-of-view: specified by (altitude degrees, azimuth degrees)
    ax.view_init(30, 0)

    #initialization function: plot the background of each frame
    def init():
        for line, pt in zip(lines, pts):
            line.set_data([], [])
            line.set_3d_properties([])

            pt.set_data([], [])
            pt.set_3d_properties([])
        return lines + pts

    #animation function.  This will be called sequentially with the frame number
    def animate(i):
        # we'll step two time-steps per frame.  This leads to nice results.
        i = (2 * i) % x_t.shape[1]

        for line, pt, xi in zip(lines, pts, x_t):
            x, y, z = xi[:i].T
            line.set_data(x, y)
            line.set_3d_properties(z)

            pt.set_data(x[-1:], y[-1:])
            pt.set_3d_properties(z[-1:])

        if rotate: ax.view_init(30, 0.3 * i)
        fig.canvas.draw()
        return lines + pts

    #instantiate the animator.
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=frames, interval=30, blit=True, repeat=False)
    #show it
    ax.set_title('Lorenz Simulation')
    ax.set_xlabel('x(t)')
    ax.set_ylabel('y(t)')
    ax.set_zlabel('z(t)')
    plt.show()

In [45]:
def euler_ode(func,y0,t,h):
    ''' y0 es 1 x n'''
    t_size = t.size
    solutions = np.zeros((t.size,y0.size))
    solutions[0] = y0 
    for i in range(1, t_size):
        solutions[i] = solutions[i-1]+h*func(t[i-1],solutions[i-1])
    return solutions

def midpoint_ode(func,y0,t,h):
    t_size = t.size
    solutions = np.zeros((t.size,y0.size))
    solutions[0, :] = y0 
    for i in range(1, t_size):
        current_t = t[i-1]
        y = solutions[i-1]
        k1 = y + h/2.0*func(current_t,y)
        solutions[i] = y + h*func(current_t + h/2.0,k1)
    return solutions

def RK4_ode(func,y0,t,h):
    t_size = t.size
    solutions = np.zeros((t.size,y0.size))
    solutions[0, :] = y0
    for i in range(1, t_size):
        current_t = t[i-1]
        y = solutions[i-1]
        
        k1=func(current_t,y)
        k2=func(current_t+h/2.0,y+(h/2.0)*k1)
   #     k2=f(t+h/2.0,y+(h/2.0)*k1)
        
        k3=func(current_t+h/2.0,y+(h/2.0)*k2)
   #     k3=f(t+h/2.0,y+(h/2.0)*k2)
        
        k4=func(current_t+h,y+h*k3)
        solutions[i] = y+(h/6.0)*(k1+2*k2+2*k3+k4)
  #  k4=f(t+h,y+h*k3)
    return solutions

In [49]:
def F(t,y):
    _y1 = y[0]
    y1 = _y1**2
    return np.array([t*_y1+t**3])

y0 = np.array([1.0])
h = 0.2
t = np.linspace(0,5*h,5)
print t
print euler_ode(F,y0,t,h)
print midpoint_ode(F,y0,t,h)
print RK4_ode(F,y0,t,h)

[ 0.    0.25  0.5   0.75  1.  ]
[[ 1.        ]
 [ 1.        ]
 [ 1.053125  ]
 [ 1.1834375 ]
 [ 1.44532813]]
[[ 1.        ]
 [ 1.0202    ]
 [ 1.10208372]
 [ 1.28564627]
 [ 1.65059501]]
[[ 1.        ]
 [ 1.02060335]
 [ 1.1041513 ]
 [ 1.2917912 ]
 [ 1.66600241]]
